In [1]:
%load_ext autoreload
%autoreload 2

Important constants

In [56]:
NUM_TERMINS = 50

Some frequently used imports

In [57]:
import numpy as np
import pandas as pd

In [58]:
from tqdm.notebook import trange
from tqdm.notebook import tqdm

Load the dataset and lemmatizer

In [59]:
from utils.data import Lemmatizer
from datasets import Dataset

In [60]:
dataset = Dataset.from_parquet("./doc.parquet")

Generating train split: 0 examples [00:00, ? examples/s]

In [61]:
lemmatizer = Lemmatizer("en")

Fit TfidfVectorizer on the dataset

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
vectorizer = TfidfVectorizer()

In [64]:
texts = []
for text in tqdm(dataset["text"]):
    texts.append(lemmatizer.lemmatize_text(text))

  0%|          | 0/20 [00:00<?, ?it/s]

In [65]:
X = vectorizer.fit_transform(texts)

Count nonzero elements

In [66]:
ans = []
for i in trange(X.shape[1]):
    ans.append(X.getcol(i).count_nonzero())

  0%|          | 0/2983 [00:00<?, ?it/s]

In [67]:
word_nonzero = np.array(ans)

Create mask for stop words 

In [68]:
import nltk
from nltk.corpus import stopwords

In [69]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\znaum\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
stop_mask = np.array([word in stopwords.words("english") for word in vectorizer.get_feature_names_out()])

Get the most frequent termins

In [71]:
df = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "count": word_nonzero * np.invert(stop_mask)
})

In [72]:
df = df.sort_values("count", ascending=False)

In [73]:
termins = df["word"][:NUM_TERMINS].to_list()

Save the termins

In [74]:
import json

In [75]:
json.dump(termins, open("./termins.json", "w"))

debug

In [3]:
from datasets import Dataset

In [48]:
class args:
    parquet = "./dataset_ua.parquet"
    lang = "uk"
    stopwords = "./stopwords/uk.json"
    num = 100
    file = "./termins.json"

In [14]:
dataset = Dataset.from_parquet(args.parquet)

In [15]:
from utils.data import Lemmatizer

In [16]:
lemmatizer = Lemmatizer(args.lang)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer()

In [27]:
from tqdm.auto import tqdm, trange

In [23]:
texts = []
for text in tqdm(dataset["text"]):
    texts.append(lemmatizer.lemmatize_text(text))

  0%|          | 0/1919 [00:00<?, ?it/s]

In [25]:
X = vectorizer.fit_transform(texts)

In [29]:
import numpy as np

In [30]:
ans = []
for i in trange(X.shape[1]):
    ans.append(X.getcol(i).count_nonzero())
word_nonzero = np.array(ans)

  0%|          | 0/31772 [00:00<?, ?it/s]

In [34]:
import json

In [35]:
if args.stopwords != "none":
    stopwords = json.load(open(args.stopwords, "r", encoding="utf-8"))
    stop_mask = np.array([word in stopwords for word in vectorizer.get_feature_names_out()])
else:
    stop_mask = np.full(len(vectorizer.get_feature_names_out()), False)

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "count": word_nonzero * np.invert(stop_mask)
})

In [45]:
df = df.sort_values("count", ascending=False)

In [46]:
df

,word,count
26991,стати,720
25082,рок,719
18526,новий,691
28798,україна,627
16366,можний,627
...,...,...
2899,аби,0
13076,каф,0
27973,тепер,0
19143,одному,0


In [53]:
termins = df["word"][:args.num].to_list()
json.dump(termins, open(args.file, "w", encoding="utf-8"))